<b><center><font size="5">DINER ANALYSIS</font></center></b>  

In [1]:
import pandas as pd

In [2]:
sales = pd.read_excel("CS1_Dataset.xlsx",sheet_name ='Sales')
menu = pd.read_excel("CS1_Dataset.xlsx",sheet_name ='Menu')
members = pd.read_excel("CS1_Dataset.xlsx",sheet_name ='Members')

In [3]:
sales.head()

,Customer_id,order_date,product_id
0,A,2021-01-01,1
1,A,2021-01-01,2
2,A,2021-01-07,2
3,A,2021-01-10,3
4,A,2021-01-11,3


In [4]:
menu.head()

,product_id,product_name,price
0,1,sushi,10
1,2,curry,15
2,3,ramen,12


In [5]:
members.head()

,Customer_id,join_date
0,A,2021-01-07
1,B,2021-01-09
2,C,2021-01-02


What is the total amount each customer spent at the restaurant?

In [6]:
sales.merge(menu, on = 'product_id' ).groupby('Customer_id', as_index = False)['price'].sum()

,Customer_id,price
0,A,76
1,B,74
2,C,36


How many days has each customer visited the restaurant?

In [7]:
sales.groupby('Customer_id', as_index = False)['order_date'].nunique()

,Customer_id,order_date
0,A,4
1,B,6
2,C,2


What was the first item from the menu purchased by each customer?

In [8]:
q3_1 = sales.merge(menu, on = 'product_id') 
q3_1['order_rank'] = q3_1.groupby('Customer_id')['order_date'].rank(method = 'first')
q3_1[['Customer_id','product_name']][q3_1['order_rank'] == 1.0]

,Customer_id,product_name
0,A,sushi
5,B,curry
12,C,ramen


What is the most purchased item on the menu and how many times was it purchased by all customers?

In [9]:
q4_1 = sales.merge(menu, on = 'product_id').groupby('product_name', as_index = False).size()
q4_1.sort_values('size',ascending = False)

,product_name,size
1,ramen,8
0,curry,4
2,sushi,3


Which item was the most popular for each customer?

In [10]:
q5_1 = sales.merge(menu , on = 'product_id').groupby(['Customer_id','product_name'], as_index = False).size()
q5_1['rnk'] = q5_1.groupby('Customer_id')['size'].rank(method = 'dense', ascending = False)
q5_1[['Customer_id','product_name']][q5_1['rnk']==1]

,Customer_id,product_name
1,A,ramen
3,B,curry
4,B,ramen
5,B,sushi
6,C,ramen


Which item was purchased first by the customer after they became a member? 

In [11]:
q6 = sales.merge(members, on = 'Customer_id').merge(menu, on = 'product_id').query('order_date > join_date')
q6.loc[q6.groupby('Customer_id')['order_date'].idxmin()]

,Customer_id,order_date,product_id,join_date,product_name,price
7,A,2021-01-10,3,2021-01-07,ramen,12
2,B,2021-01-11,1,2021-01-09,sushi,10
14,C,2021-01-07,3,2021-01-02,ramen,12


Which item was purchased just before the customer became a member?

In [12]:
q7 = sales.merge(members, on = 'Customer_id').merge(menu, on = 'product_id').query('order_date < join_date')
q7.loc[q7.groupby('Customer_id')['order_date'].idxmax()]

,Customer_id,order_date,product_id,join_date,product_name,price
0,A,2021-01-01,1,2021-01-07,sushi,10
1,B,2021-01-04,1,2021-01-09,sushi,10
12,C,2021-01-01,3,2021-01-02,ramen,12


What is the total items and amount spent for each member before they became a member?

In [13]:
sales.merge(members, on = 'Customer_id').merge(menu,on='product_id').\
query('order_date < join_date').groupby('Customer_id',as_index = True).agg({'product_id':'count','price':'sum'})

,product_id,price
Customer_id,,
A,2,25
B,3,40
C,2,24


If each $1 spent equates to 10 points and sushi has a 2x points multiplier - how many points would each customer have?

In [14]:
def points(df):
    if(df['product_name'] =='sushi'):
        return df['price']*20
    else :
        return df['price']*10
    
q9_1 = sales.merge(menu,on = 'product_id')
q9_1['points'] = q9_1.apply(points,axis = 1)
q9_1.groupby('Customer_id',as_index = False)['points'].sum()

,Customer_id,points
0,A,860
1,B,940
2,C,360


In the first week after a customer joins the program (including their join date) they earn 2x points on all items, not just sushi - how many points do customer A and B have at the end of January?

In [15]:
def points_v2(df):
    if((df['days_diff'] >=0 and df['days_diff']<=7) or (df['product_name'] =='sushi')) :
        return df['price']*20
    else :
        return df['price']*10
q10 = sales.merge(members, on = 'Customer_id').merge(menu,on='product_id')
q10['days_diff'] = (q10['order_date']-q10['join_date']).dt.days
q10['points'] = q10.apply(points_v2,axis =1)
q10[(q10['Customer_id'].isin(['A','B'])) & (q10['order_date'] < '2021-02-01')].\
groupby('Customer_id', as_index = False)['points'].sum()

,Customer_id,points
0,A,1370
1,B,940
